In [1]:
###################################
#GEE to analysis practice: Eliburn#
#Maintainer: Christopher Chan     #
#Date: 2024-06-04                 #
#Version: 0.0.3                   #
###################################

import io
import os, sys
import pathlib
import folium
import ee
import geemap
import pysal
import pygis
import google.auth
import geemap

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime as dt
from osgeo import gdal
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload

ee.Authenticate()
ee.Initialize()
geemap.ee_initialize()

# set paths
data_path = pathlib.Path("../data")
docs_path = pathlib.Path("../docs")

In [4]:
# Image request specifications
S2_coll = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

AOI = ee.Geometry.Polygon([
  [-3.5623692655567663,55.89846606313462],
  [-3.5489796781544225,55.89870667205676],
  [-3.5458039426807897,55.89062139493942],
  [-3.56751910686536,55.891632146720085],
  [-3.5623692655567663,55.89846606313462],
  ])

startdate = str(dt(2023, 9, 1).date())
todaydt = str(dt.today())
enddate = str(dt.today().date())

# Cloud mask (OMITS date:start timestamp, obsolete from Feb 2022?)
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

vis = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

filteredCollection = (ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
  .filterBounds(AOI)
  .filterDate(startdate, enddate)
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
  #.map(mask_s2_clouds)
  )

def clip_AOI(image):
  return image.clip(AOI)

clipmaskCollection = filteredCollection.map(clip_AOI)
clipmask_median = clipmaskCollection.median()

AOI_centre = AOI.centroid().coordinates().getInfo()

# Create raster folder in 01_raw if not exist
if not os.path.exists(pathlib.Path(f"{data_path}/01_raw/raster")):
  os.makedirs(pathlib.Path(f"{data_path}/01_raw/raster"))
elif not os.path.exists(pathlib.Path(f"{data_path}/01_raw/vector")):
  os.makedirs(pathlib.Path(f"{data_path}/01_raw/vector"))
else:
  print("Geospatial data folder exist for 01_raw")

Geospatial data folder exist for 01_raw


In [5]:
# Download to google drive
nimg = clipmaskCollection.toList(clipmaskCollection.size().getInfo()).size().getInfo()

for i in range(nimg):
    img = ee.Image(clipmaskCollection.toList(nimg).get(i))
    reproj_img = img.reproject(crs="EPSG:27700", scale=10)
    date = reproj_img.date().format('YYYY-MM-dd').getInfo()

    task = ee.batch.Export.image.toDrive(reproj_img.toFloat(),
                                          description=date,
                                          folder=f'L2A_Eliburn_{todaydt}',
                                          fileNamePrefix= f"L2A_Eliburn_{date}",
                                          crs='EPSG:27700',
                                          region = AOI,
                                          scale = 10,
                                          fileFormat = 'GeoTIFF',)
    task.start()


In [6]:
Map = geemap.Map()
Map.setCenter(AOI_centre[0], AOI_centre[1], 15)
Map.addLayer(clipmask_median, vis, "S2: L1C median")

Map

Map(center=[55.894552363344935, -3.5559986968427935], controls=(WidgetControl(options=['position', 'transparen…

In [13]:
# Download from google drive
def download_file(real_file_id):
  """Downloads a file
  Args:
      real_file_id: ID of the file to download
  Returns : IO object with location.

  Load pre-authorized user credentials from the environment.
  TODO(developer) - See https://developers.google.com/identity
  for guides on implementing OAuth2 for the application.
  """
  creds, _ = google.auth.default()

  try:
    # create drive api client
    service = build("drive", "v3", credentials=creds)

    file_id = real_file_id

    # pylint: disable=maybe-no-member
    request = service.files().get_media(fileId=file_id)
    file = io.BytesIO()
    downloader = MediaIoBaseDownload(file, request)
    done = False
    while done is False:
      status, done = downloader.next_chunk()
      print(f"Download {int(status.progress() * 100)}.")

  except HttpError as error:
    print(f"An error occurred: {error}")
    file = None 

  return file.getvalue()

download_file(real_file_id = str(input("Enter the folder ID from google drive: ")))

An error occurred: <HttpError 403 when requesting https://www.googleapis.com/drive/v3/files/%2F1Qb69M8nL1o-EcZijkhZBev_v8_Fam4-6?alt=media returned "Request had insufficient authentication scopes.". Details: "[{'message': 'Insufficient Permission', 'domain': 'global', 'reason': 'insufficientPermissions'}]">


AttributeError: 'NoneType' object has no attribute 'getvalue'